# Динамика показателей тематических каналов в течение дня
Пример расчета охватов тематических каналов в разбивке по часам

## Описание задачи и условий расчета
- Период: 01.01.2022 - 31.12.2022
- Временной интервал: 05:00-29:00 с разбивкой по часам
- ЦА: 4+
- Место просмотра: Все места (дом+дача)
- Каналы: ПЕРВЫЙ КОСМИЧЕСКИЙ, РЫЖИЙ, ЗАГОРОДНЫЙ								
- Статистики: AvReach000, AvReach%, AvWReach000, AvWReach%, AvMReach000, AvMReach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телекомпаний получим id нужных нам каналов
cats.get_tv_company(name=['ПЕРВЫЙ КОСМИЧЕСКИЙ','РЫЖИЙ','ЗАГОРОДНЫЙ'])

# Это id: 4541, 5457, 6934

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'.
# Минимум 28 дней
date_filter = [('2022-01-01', '2022-12-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал - минимум 60 минут
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = 'tvCompanyId IN (4541, 6934, 5457)'

# Указываем список статистик для расчета
statistics = ['AvReach000','AvReachPer','AvWkReach000','AvWkReachPer','AvMReach000','AvMReachPer']

# Указываем срезы
slices = [
    'tvCompanyName', #телекомпания
    'timeBand60' #час
]

# Задаем условия сортировки: телекомпания (от а до я), временной интервал (по возр.)
sortings = {'tvCompanyName':'ASC', 'timeBand60':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia 
    "useNbd": False #Расчет накопленного охвата без nbd коррекции
}

## Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
#меняем формат поля с временными интервалами для корректной сортировки
df = df.astype(dtype={'timeBand60':int})

# Формируем сводную таблицу: строки - временной интервал, столбцы - каналы, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['timeBand60'], 
                        columns=['tvCompanyName'])
df

In [ ]:
# Опционально: поменять местами уровни в столбцах
df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('thematic_02_dynamics_during_the_day')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)